# Game of Zones: Using ML to Predict the Outcome of an MLB At Bat

Goal: predict where the batter is most likely to hit the ball (zones of the field) in an at-bat given the situation and the pitcher he is facing
    
Input Data: 
- the pitcher's repertoire: given that each pitcher has a different arsenal of pitches and each pitch moves differently, we use a cluster analysis to categorize pitch types.  In this way, we put each pitcher on the same footing.

- pitcher stats such as groundball and flyball rates

- the game situation: the inning (and top/bottom), the number of outs, positions of baserunners, the count, positions of fielders(?)

- the batter's priors: distribution of batted balls into zones

- any other batter data?

Output: 
- probabilities for each zone on the field where the batter can hit the ball

- contributing factors for each prediction (things the defensive team could use to intervene)

In [12]:
%load_ext autoreload
%autoreload 2
from pybaseball import statcast
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from matplotlib import patches
%matplotlib inline

from get_data import get_data, get_hit_zone, get_pitch_data, get_situation_data
from pitch_clustering import pitch_clustering
from batter_zone import get_batter_zone_data

# use Statcast data (from 2015-2018) so we can get spin rate, etc.
train_data_dates = [('2015-04-05', '2015-10-04')] #,      # 2015 data
#                     ('2016-04-03', '2016-10-02'),       # 2016 data
#                     ('2017-04-02', '2017-10-01'),       # 2017 data
#                     ('2018-03-29', '2018-10-01')]       # 2018 data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Build the Outcome

In [2]:
# get the outcome data
outcome_data = get_data(get_hit_zone, train_data_dates)

# write to file
outcome_data.to_csv("./outcome.csv", index=False)

print(f"Shape of the outcome data: {outcome_data.shape}")
outcome_data.head()

This is a large query, it may take a moment to complete
Shape of the outcome data: (87199, 5)


,game_pk,index,batter,pitcher,hit_zone
0,416079,247,150029,544727,4
2,416079,262,547180,544727,2
10,416079,341,543685,544727,1
15,416079,404,502517,595014,2
24,416079,519,434158,595014,4


### Build the Pitcher Data

In [ ]:
# get the outcome data
pitch_data = get_data(get_pitch_data, train_data_dates)

# write to file
pitch_data.to_csv("./pitch.csv", index=False)

# print the number of pitchers in the data set
print(f"Number of pitchers in the data: {len(pitch_data['pitcher'].unique())}")

print(f"Shape of training data: {pitch_data.shape}")

pitch_data.head()

This is a large query, it may take a moment to complete


### Build the Batter's Prior Zone Distribution

In [8]:
# use the hit zone data from the outcome (calculated above)
batter_zone_data = pd.read_csv("./outcome.csv")

batter_zone_data_pct = get_batter_zone_data(batter_zone_data)

batter_zone_data_pct.to_csv("./batter_zones.csv", index=False)

print(batter_zone_data_pct.shape)
batter_zone_data_pct.head()

(865, 5)


,batter,batter_zone_1,batter_zone_2,batter_zone_3,batter_zone_4
0,112526,0.105263,0.421053,0.263158,0.210526
1,116338,0.114094,0.278523,0.342282,0.265101
2,120074,0.249240,0.024316,0.416413,0.310030
3,121347,0.056738,0.319149,0.283688,0.340426
4,133380,0.061290,0.232258,0.367742,0.338710


### Get the Game Situation Features

In [13]:
situation_data = get_data(get_situation_data, train_data_dates)

# write to file
situation_data.to_csv("./situation.csv", index=False)

print(situation_data.shape)
situation_data.head()

This is a large query, it may take a moment to complete
(101234, 14)


,game_pk,index,batter,pitcher,balls,strikes,outs_when_up,inning,on_1b,on_2b,on_3b,bat_right,pitch_right,score_diff
0,416079,247,150029,544727,0.0,1.0,2.0,9.0,False,True,False,True,True,-1.0
2,416079,262,547180,544727,0.0,0.0,2.0,9.0,False,False,False,False,True,-1.0
10,416079,341,543685,544727,3.0,1.0,0.0,9.0,False,False,False,True,True,-1.0
15,416079,404,502517,595014,2.0,1.0,1.0,8.0,True,False,False,False,True,1.0
24,416079,519,434158,595014,2.0,1.0,1.0,8.0,False,False,False,False,True,0.0


### Combine the Game Situation, Pitcher and Batter Features along with the Outcome

In [ ]:
game_situation_df = pd.read_csv("./situation.csv")

pitch_type_df = pd.read_csv("./pitch_types.csv")
pitch_type_df.drop('player_name', axis=1, inplace=True)
pitch_type_df['pitcher'] = pitch_type_df['pitcher'].astype(int)

batter_zone_df = pd.read_csv("./batter_zones.csv")

outcome_df = pd.read_csv("/home/chris/data/baseball/HitZone/train_4zones.csv")

full_data = pd.merge(game_situation_df, pitch_type_df, on="pitcher")
full_data = pd.merge(full_data, batter_zone_df, on="batter")
full_data = pd.merge(outcome_df, full_data, on=['game_pk', 'index', 'batter', 'pitcher'])

print(full_data.shape)
full_data.head()

In [ ]:
full_data = full_data.drop(['game_pk', 'index', 'batter', 'pitcher'], axis=1)

# split the dataframe into a feature set and an outcome column
X = full_data.drop('hit_zone', axis=1)
y = full_data['hit_zone']

# split the data into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4256)

In [ ]:
# ----------------------
# train an XGBoost model
# ----------------------

# small set of hyperparameters to optimize over
xgb_params = {"max_depth": (3, 5, 10, 15, 20),
              "learning_rate": (0.01, 0.5, 0.1, 0.2, 0.4),
              "gamma": (0, 33, 66, 100),
              "min_child_weight": (0, 33, 66, 100),
              "colsample_bytree": (0.5, 0.75, 1),
              "subsample": (0.5, 0.75, 1),}

# perform the paramater grid search using 5-fold cross validation
xgb_opt = GridSearchCV(XGBClassifier(objective='multi:softprob', num_class=4), 
                       param_grid=xgb_params, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

#xgb_opt = XGBClassifier(objective='multi:softprob', num_class=4)

# perform fit and make predictions
xgb_opt.fit(X_train, y_train)
y_pred = xgb_opt.predict(X_test)
y_prob = xgb_opt.predict_proba(X_test)

# compute accuracy
accuracy = round(accuracy_score(y_test, y_pred) * 100, 1)

# print the confusion matrix
print(confusion_matrix(y_test, y_pred))

In [ ]:
accuracy

In [ ]:
features = X_train.columns.tolist()
importances = list(xgb_opt.feature_importances_)
for i in range(len(features)):
    print(features[i] + "\t" + str(importances[i] * 100.))